### import libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, SimpleRNN, TimeDistributed, LSTM, GRU, Bidirectional
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

### Data load

In [2]:
Data = np.load('Data/EpilepsyData.npz', allow_pickle=True)

In [3]:
Train_x = Data['Train_x']
Train_y = Data['Train_y']
Test_x = Data['Test_x']
Test_y = Data['Test_y']

In [4]:
Train_x

array([[[ 0.35,  0.22,  0.55],
        [ 0.4 ,  0.01,  0.51],
        [ 0.44, -0.71,  0.52],
        ...,
        [ 0.62,  0.72,  0.5 ],
        [ 0.65, -0.9 ,  0.55],
        [ 0.69, -1.3 ,  0.54]],

       [[-0.91, -0.05,  0.06],
        [-0.92, -0.02, -1.57],
        [-0.95,  0.01, -0.99],
        ...,
        [-0.47,  0.88, -0.95],
        [-0.45,  0.8 , -0.83],
        [-0.45,  0.76, -0.78]],

       [[ 0.32,  1.  ,  0.37],
        [ 0.71, -1.33,  0.37],
        [ 0.7 , -1.44,  0.37],
        ...,
        [ 0.59, -1.04,  0.45],
        [ 0.59,  0.47, -0.64],
        [ 0.59,  1.23,  0.66]],

       ...,

       [[ 1.16,  0.9 , -1.03],
        [-0.02,  0.59, -1.11],
        [-1.04,  0.52, -0.88],
        ...,
        [-0.51,  0.29, -0.02],
        [-0.94,  0.29, -0.02],
        [-0.48,  0.29, -0.02]],

       [[-0.74, -0.19, -0.14],
        [-0.14, -0.24, -0.26],
        [ 0.12, -0.34, -0.27],
        ...,
        [-0.4 , -0.54, -0.45],
        [-0.23, -0.43, -0.86],
        [-0.27,

In [5]:
Train_x.dtype

dtype('float64')

In [6]:
Train_x.shape

(137, 206, 3)

In [7]:
TimeRange = Train_x.shape[1]
nFeatures = Train_x.shape[2]

### Data Shuffle

In [9]:
arr = np.arange(Train_x.shape[0])
np.random.shuffle(arr)
arr

array([ 63, 123, 121,  48,  93,  20,  74,  41, 116, 132,   8, 108,  70,
        76, 103, 118,  34,  53, 122,  11, 110,  56,   3, 127,  33,  86,
        26, 133, 125,  64,   2,  94,  21, 131, 100,  96, 119, 106,  39,
       112,  89,  15,  91,  82,   7,  36,  98, 124,  14,  35,  31,  88,
         1,  47,  73,  69,  97, 105, 109,  40,  84,  38,  46,  80,  30,
        43,  83, 120,  99,  55, 107, 129,  42,  79,  85,  61,  81,  37,
        60,  71,  12,  78,  23,  13,  52,  65,  77,  25, 111,  68,  18,
        58,  92,  19,   5,  72,  90, 134,  51,  22,  62, 101,  10, 130,
        59, 102, 126,   4,  66,   6,  57,  44,   0, 117, 115, 135,   9,
        29,  17,  24,  49,  87,  67, 128,  75,  95,  54,  50, 114, 136,
        28,  32,  45,  27,  16, 113, 104])

In [10]:
Train_x = Train_x[arr]
Train_y = Train_y[arr]

In [11]:
arr = np.arange(Test_x.shape[0])
np.random.shuffle(arr)

Test_x = Test_x[arr]
Test_y = Test_y[arr]

### Model structure with Simple RNN

In [15]:
model = Sequential()

# RNN. 현재 RNN은 return_sequences (모든 x time-step에 대하여 output을 냄)
model.add(SimpleRNN(24, activation='linear', return_sequences = True, input_shape=( TimeRange, nFeatures,  )))
model.add(SimpleRNN(24, activation='linear', return_sequences = False, input_shape=( TimeRange, nFeatures,  )))
model.add(Dense(4, activation='softmax'))

In [16]:
model.compile(loss = "categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_6 (SimpleRNN)    (None, 206, 24)           672       
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 24)                1176      
                                                                 
 dense_3 (Dense)             (None, 4)                 100       
                                                                 
Total params: 1,948
Trainable params: 1,948
Non-trainable params: 0
_________________________________________________________________


### Call-back function for fine-tunning

In [17]:
# Call-back 함수
# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath='MODEL/{epoch:03d}-{loss:.4f}-{val_loss:.4f}.hdf5',
            monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#minitor -> loss -> val_loss

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, verbose=1, min_lr=1e-8)
# factor: Learning rate에 곱할 것.
#0.1 -> 0.08 -> 0.064 ....
#monitor='loss' -> monitor='val_loss'

CALLBACK = [CP, LR]

In [18]:
history_finetunning = model.fit(
    Train_x, Train_y,
    epochs=300,
    batch_size=52,
    callbacks=CALLBACK,
    validation_data=(Test_x, Test_y),
    shuffle=True)

Epoch 1/300
3/3 [==============================] - ETA: 0s - loss: 38.7180 - accuracy: 0.2190
Epoch 1: val_loss improved from inf to 12.37317, saving model to MODEL\001-38.7180-12.3732.hdf5
3/3 [==============================] - 4s 552ms/step - loss: 38.7180 - accuracy: 0.2190 - val_loss: 12.3732 - val_accuracy: 0.3406 - lr: 0.0010
Epoch 2/300
3/3 [==============================] - ETA: 0s - loss: 13.8104 - accuracy: 0.4161
Epoch 2: val_loss improved from 12.37317 to 9.74479, saving model to MODEL\002-13.8104-9.7448.hdf5
3/3 [==============================] - 0s 175ms/step - loss: 13.8104 - accuracy: 0.4161 - val_loss: 9.7448 - val_accuracy: 0.3261 - lr: 0.0010
Epoch 3/300
3/3 [==============================] - ETA: 0s - loss: 7.9136 - accuracy: 0.3285
Epoch 3: val_loss improved from 9.74479 to 7.17544, saving model to MODEL\003-7.9136-7.1754.hdf5
3/3 [==============================] - 1s 203ms/step - loss: 7.9136 - accuracy: 0.3285 - val_loss: 7.1754 - val_accuracy: 0.2826 - lr: 0.001

3/3 [==============================] - 0s 172ms/step - loss: 1.7292 - accuracy: 0.4307 - val_loss: 2.8575 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 26/300
3/3 [==============================] - ETA: 0s - loss: 1.6879 - accuracy: 0.4599
Epoch 26: val_loss improved from 2.85752 to 2.83434, saving model to MODEL\026-1.6879-2.8343.hdf5
3/3 [==============================] - 0s 175ms/step - loss: 1.6879 - accuracy: 0.4599 - val_loss: 2.8343 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 27/300
3/3 [==============================] - ETA: 0s - loss: 1.6521 - accuracy: 0.4526
Epoch 27: val_loss did not improve from 2.83434
3/3 [==============================] - 0s 165ms/step - loss: 1.6521 - accuracy: 0.4526 - val_loss: 2.8449 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 28/300
3/3 [==============================] - ETA: 0s - loss: 1.6169 - accuracy: 0.4672
Epoch 28: val_loss improved from 2.83434 to 2.82681, saving model to MODEL\028-1.6169-2.8268.hdf5
3/3 [==============================] - 1s 224ms/

Epoch 52/300
3/3 [==============================] - ETA: 0s - loss: 1.0695 - accuracy: 0.6350
Epoch 52: val_loss improved from 2.60699 to 2.58184, saving model to MODEL\052-1.0695-2.5818.hdf5
3/3 [==============================] - 0s 159ms/step - loss: 1.0695 - accuracy: 0.6350 - val_loss: 2.5818 - val_accuracy: 0.3696 - lr: 0.0010
Epoch 53/300
3/3 [==============================] - ETA: 0s - loss: 1.0553 - accuracy: 0.6423
Epoch 53: val_loss improved from 2.58184 to 2.57468, saving model to MODEL\053-1.0553-2.5747.hdf5
3/3 [==============================] - 0s 153ms/step - loss: 1.0553 - accuracy: 0.6423 - val_loss: 2.5747 - val_accuracy: 0.3696 - lr: 0.0010
Epoch 54/300
3/3 [==============================] - ETA: 0s - loss: 1.0452 - accuracy: 0.6423
Epoch 54: val_loss did not improve from 2.57468
3/3 [==============================] - 0s 139ms/step - loss: 1.0452 - accuracy: 0.6423 - val_loss: 2.5788 - val_accuracy: 0.3768 - lr: 0.0010
Epoch 55/300
3/3 [==============================

Epoch 80/300
3/3 [==============================] - ETA: 0s - loss: 0.8208 - accuracy: 0.6861
Epoch 80: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 131ms/step - loss: 0.8208 - accuracy: 0.6861 - val_loss: 2.5943 - val_accuracy: 0.4203 - lr: 8.0000e-04
Epoch 81/300
3/3 [==============================] - ETA: 0s - loss: 0.8148 - accuracy: 0.6788
Epoch 81: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 174ms/step - loss: 0.8148 - accuracy: 0.6788 - val_loss: 2.5698 - val_accuracy: 0.4420 - lr: 8.0000e-04
Epoch 82/300
3/3 [==============================] - ETA: 0s - loss: 0.8129 - accuracy: 0.6861
Epoch 82: val_loss did not improve from 2.51422

Epoch 82: ReduceLROnPlateau reducing learning rate to 0.0006400000303983689.
3/3 [==============================] - 0s 176ms/step - loss: 0.8129 - accuracy: 0.6861 - val_loss: 2.5828 - val_accuracy: 0.4275 - lr: 8.0000e-04
Epoch 83/300
3/3 [==============================] - ETA: 0

Epoch 108/300
3/3 [==============================] - ETA: 0s - loss: 0.7102 - accuracy: 0.7372
Epoch 108: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 127ms/step - loss: 0.7102 - accuracy: 0.7372 - val_loss: 2.7366 - val_accuracy: 0.4203 - lr: 4.0960e-04
Epoch 109/300
3/3 [==============================] - ETA: 0s - loss: 0.7049 - accuracy: 0.7372
Epoch 109: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 144ms/step - loss: 0.7049 - accuracy: 0.7372 - val_loss: 2.7543 - val_accuracy: 0.4203 - lr: 4.0960e-04
Epoch 110/300
3/3 [==============================] - ETA: 0s - loss: 0.7036 - accuracy: 0.7299
Epoch 110: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 139ms/step - loss: 0.7036 - accuracy: 0.7299 - val_loss: 2.7781 - val_accuracy: 0.4058 - lr: 4.0960e-04
Epoch 111/300
3/3 [==============================] - ETA: 0s - loss: 0.7053 - accuracy: 0.7226
Epoch 111: val_loss did not improve

...

Epoch 163/300
3/3 [==============================] - ETA: 0s - loss: 0.6231 - accuracy: 0.7664
Epoch 163: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 187ms/step - loss: 0.6231 - accuracy: 0.7664 - val_loss: 2.9616 - val_accuracy: 0.4058 - lr: 1.0737e-04
Epoch 164/300
3/3 [==============================] - ETA: 0s - loss: 0.6231 - accuracy: 0.7737
Epoch 164: val_loss did not improve from 2.51422
3/3 [==============================] - 1s 192ms/step - loss: 0.6231 - accuracy: 0.7737 - val_loss: 2.9656 - val_accuracy: 0.4130 - lr: 1.0737e-04
Epoch 165/300
3/3 [==============================] - ETA: 0s - loss: 0.6210 - accuracy: 0.7591
Epoch 165: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 140ms/step - loss: 0.6210 - accuracy: 0.7591 - val_loss: 2.9638 - val_accuracy: 0.4058 - lr: 1.0737e-04
Epoch 166/300
3/3 [==============================] - ETA: 0s - loss: 0.6197 - accuracy: 0.7518
Epoch 166: val_loss did not improve

Epoch 191/300
3/3 [==============================] - ETA: 0s - loss: 0.6059 - accuracy: 0.7664
Epoch 191: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 177ms/step - loss: 0.6059 - accuracy: 0.7664 - val_loss: 3.0109 - val_accuracy: 0.4203 - lr: 6.8719e-05
Epoch 192/300
3/3 [==============================] - ETA: 0s - loss: 0.6041 - accuracy: 0.7664
Epoch 192: val_loss did not improve from 2.51422

Epoch 192: ReduceLROnPlateau reducing learning rate to 5.497558740898967e-05.
3/3 [==============================] - 0s 175ms/step - loss: 0.6041 - accuracy: 0.7664 - val_loss: 3.0085 - val_accuracy: 0.4203 - lr: 6.8719e-05
Epoch 193/300
3/3 [==============================] - ETA: 0s - loss: 0.6034 - accuracy: 0.7591
Epoch 193: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 176ms/step - loss: 0.6034 - accuracy: 0.7591 - val_loss: 3.0092 - val_accuracy: 0.4203 - lr: 5.4976e-05
Epoch 194/300
3/3 [==============================] 

3/3 [==============================] - 0s 124ms/step - loss: 0.5951 - accuracy: 0.7737 - val_loss: 3.0417 - val_accuracy: 0.4203 - lr: 3.5184e-05
Epoch 219/300
3/3 [==============================] - ETA: 0s - loss: 0.5950 - accuracy: 0.7737
Epoch 219: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 124ms/step - loss: 0.5950 - accuracy: 0.7737 - val_loss: 3.0395 - val_accuracy: 0.4203 - lr: 3.5184e-05
Epoch 220/300
3/3 [==============================] - ETA: 0s - loss: 0.5948 - accuracy: 0.7737
Epoch 220: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 173ms/step - loss: 0.5948 - accuracy: 0.7737 - val_loss: 3.0402 - val_accuracy: 0.4203 - lr: 3.5184e-05
Epoch 221/300
3/3 [==============================] - ETA: 0s - loss: 0.5940 - accuracy: 0.7737
Epoch 221: val_loss did not improve from 2.51422
3/3 [==============================] - 1s 208ms/step - loss: 0.5940 - accuracy: 0.7737 - val_loss: 3.0444 - val_accuracy: 0.4203 -

Epoch 246/300
3/3 [==============================] - ETA: 0s - loss: 0.5890 - accuracy: 0.7737
Epoch 246: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 128ms/step - loss: 0.5890 - accuracy: 0.7737 - val_loss: 3.0564 - val_accuracy: 0.4203 - lr: 1.8014e-05
Epoch 247/300
3/3 [==============================] - ETA: 0s - loss: 0.5889 - accuracy: 0.7737
Epoch 247: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 130ms/step - loss: 0.5889 - accuracy: 0.7737 - val_loss: 3.0583 - val_accuracy: 0.4203 - lr: 1.8014e-05
Epoch 248/300
3/3 [==============================] - ETA: 0s - loss: 0.5888 - accuracy: 0.7737
Epoch 248: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 129ms/step - loss: 0.5888 - accuracy: 0.7737 - val_loss: 3.0588 - val_accuracy: 0.4203 - lr: 1.8014e-05
Epoch 249/300
3/3 [==============================] - ETA: 0s - loss: 0.5885 - accuracy: 0.7810
Epoch 249: val_loss did not improve

3/3 [==============================] - 0s 181ms/step - loss: 0.5858 - accuracy: 0.7737 - val_loss: 3.0608 - val_accuracy: 0.4203 - lr: 9.2234e-06
Epoch 274/300
3/3 [==============================] - ETA: 0s - loss: 0.5859 - accuracy: 0.7737
Epoch 274: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 181ms/step - loss: 0.5859 - accuracy: 0.7737 - val_loss: 3.0620 - val_accuracy: 0.4203 - lr: 9.2234e-06
Epoch 275/300
3/3 [==============================] - ETA: 0s - loss: 0.5857 - accuracy: 0.7737
Epoch 275: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 177ms/step - loss: 0.5857 - accuracy: 0.7737 - val_loss: 3.0622 - val_accuracy: 0.4203 - lr: 9.2234e-06
Epoch 276/300
3/3 [==============================] - ETA: 0s - loss: 0.5857 - accuracy: 0.7737
Epoch 276: val_loss did not improve from 2.51422
3/3 [==============================] - 0s 173ms/step - loss: 0.5857 - accuracy: 0.7737 - val_loss: 3.0626 - val_accuracy: 0.4203 -

In [19]:
res = model.predict(Test_x)

5/5 [==============================] - 1s 31ms/step


In [20]:
res.argmax(axis=1)

array([3, 0, 0, 0, 3, 3, 1, 3, 1, 0, 0, 1, 1, 0, 0, 3, 0, 2, 3, 1, 0, 2,
       0, 1, 2, 1, 2, 3, 2, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 3, 2, 3, 2,
       2, 3, 1, 2, 2, 1, 0, 1, 2, 3, 1, 1, 2, 0, 1, 3, 2, 1, 2, 0, 3, 0,
       2, 1, 0, 2, 0, 2, 1, 1, 0, 2, 2, 2, 1, 1, 1, 3, 0, 3, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 2, 0, 1, 0, 0, 0, 1, 1, 1, 3, 3, 0, 2, 3, 2, 0,
       3, 2, 3, 0, 0, 3, 3, 1, 1, 3, 2, 3, 0, 3, 1, 3, 1, 1, 3, 2, 2, 3,
       0, 3, 3, 3, 0, 3], dtype=int64)

In [21]:
Test_y.argmax(axis=1)

array([1, 3, 0, 3, 0, 0, 1, 3, 0, 0, 0, 1, 3, 0, 0, 0, 3, 1, 1, 1, 3, 0,
       3, 1, 2, 0, 0, 3, 0, 1, 0, 0, 1, 0, 1, 3, 1, 3, 3, 2, 3, 2, 2, 0,
       2, 1, 3, 2, 0, 1, 3, 1, 2, 0, 2, 1, 1, 0, 2, 3, 2, 2, 3, 3, 3, 2,
       0, 1, 1, 2, 0, 2, 2, 1, 1, 2, 1, 2, 0, 1, 1, 3, 2, 3, 1, 0, 2, 1,
       1, 3, 1, 3, 1, 3, 0, 2, 0, 2, 2, 1, 3, 3, 1, 2, 3, 2, 0, 3, 2, 2,
       3, 1, 1, 1, 2, 3, 3, 3, 2, 3, 0, 3, 2, 1, 0, 0, 0, 1, 3, 3, 0, 3,
       2, 0, 1, 3, 0, 1], dtype=int64)

In [20]:
pd.Series(res.argmax(axis=1) == Test_y.argmax(axis=1)).value_counts()

False    84
True     54
dtype: int64

In [22]:
54/84+54

54.642857142857146

# LSTM
### lstm long short term memory 

In [23]:
model = Sequential()

# RNN. 현재 RNN은 return_sequences (모든 x time-step에 대하여 output을 냄)
model.add(LSTM(28, dropout=0.3, recurrent_dropout=0.3, return_sequences = True, input_shape=( TimeRange, nFeatures,  )))
model.add(LSTM(14, dropout=0.3, recurrent_dropout=0.3, return_sequences = False))
model.add(Dense(4, activation='softmax'))

In [24]:
model.compile(loss = "categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 206, 28)           3584      
                                                                 
 lstm_1 (LSTM)               (None, 14)                2408      
                                                                 
 dense_4 (Dense)             (None, 4)                 60        
                                                                 
Total params: 6,052
Trainable params: 6,052
Non-trainable params: 0
_________________________________________________________________


In [25]:
history_finetunning = model.fit(
    Train_x, Train_y,
    epochs=300,
    batch_size=52,
    callbacks=CALLBACK,
    validation_data=(Test_x, Test_y),
    shuffle=True)

Epoch 1/300
3/3 [==============================] - ETA: 0s - loss: 1.3717 - accuracy: 0.3066
Epoch 1: val_loss improved from 2.51422 to 1.37415, saving model to MODEL\001-1.3717-1.3742.hdf5
3/3 [==============================] - 10s 797ms/step - loss: 1.3717 - accuracy: 0.3066 - val_loss: 1.3742 - val_accuracy: 0.2754 - lr: 0.0010
Epoch 2/300
3/3 [==============================] - ETA: 0s - loss: 1.3616 - accuracy: 0.3139
Epoch 2: val_loss improved from 1.37415 to 1.36361, saving model to MODEL\002-1.3616-1.3636.hdf5
3/3 [==============================] - 1s 472ms/step - loss: 1.3616 - accuracy: 0.3139 - val_loss: 1.3636 - val_accuracy: 0.3043 - lr: 0.0010
Epoch 3/300
3/3 [==============================] - ETA: 0s - loss: 1.3512 - accuracy: 0.3869
Epoch 3: val_loss improved from 1.36361 to 1.35346, saving model to MODEL\003-1.3512-1.3535.hdf5
3/3 [==============================] - 1s 524ms/step - loss: 1.3512 - accuracy: 0.3869 - val_loss: 1.3535 - val_accuracy: 0.3478 - lr: 0.0010
Epo

Epoch 26/300
3/3 [==============================] - ETA: 0s - loss: 0.9547 - accuracy: 0.6058
Epoch 26: val_loss improved from 0.99804 to 0.97674, saving model to MODEL\026-0.9547-0.9767.hdf5
3/3 [==============================] - 1s 445ms/step - loss: 0.9547 - accuracy: 0.6058 - val_loss: 0.9767 - val_accuracy: 0.5797 - lr: 0.0010
Epoch 27/300
3/3 [==============================] - ETA: 0s - loss: 0.9222 - accuracy: 0.6423
Epoch 27: val_loss improved from 0.97674 to 0.96389, saving model to MODEL\027-0.9222-0.9639.hdf5
3/3 [==============================] - 1s 437ms/step - loss: 0.9222 - accuracy: 0.6423 - val_loss: 0.9639 - val_accuracy: 0.6087 - lr: 0.0010
Epoch 28/300
3/3 [==============================] - ETA: 0s - loss: 0.9100 - accuracy: 0.6350
Epoch 28: val_loss improved from 0.96389 to 0.95952, saving model to MODEL\028-0.9100-0.9595.hdf5
3/3 [==============================] - 1s 429ms/step - loss: 0.9100 - accuracy: 0.6350 - val_loss: 0.9595 - val_accuracy: 0.5942 - lr: 0.001

Epoch 53/300
3/3 [==============================] - ETA: 0s - loss: 0.7118 - accuracy: 0.7372
Epoch 53: val_loss did not improve from 0.85602
3/3 [==============================] - 1s 442ms/step - loss: 0.7118 - accuracy: 0.7372 - val_loss: 0.8629 - val_accuracy: 0.6304 - lr: 0.0010
Epoch 54/300
3/3 [==============================] - ETA: 0s - loss: 0.7397 - accuracy: 0.7226
Epoch 54: val_loss did not improve from 0.85602
3/3 [==============================] - 1s 425ms/step - loss: 0.7397 - accuracy: 0.7226 - val_loss: 0.8708 - val_accuracy: 0.6014 - lr: 0.0010
Epoch 55/300
3/3 [==============================] - ETA: 0s - loss: 0.7533 - accuracy: 0.6934
Epoch 55: val_loss improved from 0.85602 to 0.85097, saving model to MODEL\055-0.7533-0.8510.hdf5
3/3 [==============================] - 2s 532ms/step - loss: 0.7533 - accuracy: 0.6934 - val_loss: 0.8510 - val_accuracy: 0.6014 - lr: 0.0010
Epoch 56/300
3/3 [==============================] - ETA: 0s - loss: 0.7457 - accuracy: 0.7080
Epoc

3/3 [==============================] - 1s 423ms/step - loss: 0.6200 - accuracy: 0.7591 - val_loss: 0.7894 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 81/300
3/3 [==============================] - ETA: 0s - loss: 0.5797 - accuracy: 0.7810
Epoch 81: val_loss did not improve from 0.73436
3/3 [==============================] - 1s 427ms/step - loss: 0.5797 - accuracy: 0.7810 - val_loss: 0.8026 - val_accuracy: 0.6377 - lr: 0.0010
Epoch 82/300
3/3 [==============================] - ETA: 0s - loss: 0.6368 - accuracy: 0.7445
Epoch 82: val_loss did not improve from 0.73436

Epoch 82: ReduceLROnPlateau reducing learning rate to 0.000800000037997961.
3/3 [==============================] - 1s 485ms/step - loss: 0.6368 - accuracy: 0.7445 - val_loss: 0.8313 - val_accuracy: 0.6014 - lr: 0.0010
Epoch 83/300
3/3 [==============================] - ETA: 0s - loss: 0.5718 - accuracy: 0.7518
Epoch 83: val_loss did not improve from 0.73436
3/3 [==============================] - 1s 419ms/step - loss: 0.5718 - a

Epoch 108/300
3/3 [==============================] - ETA: 0s - loss: 0.4964 - accuracy: 0.8102
Epoch 108: val_loss did not improve from 0.70019
3/3 [==============================] - 1s 399ms/step - loss: 0.4964 - accuracy: 0.8102 - val_loss: 0.7357 - val_accuracy: 0.7029 - lr: 6.4000e-04
Epoch 109/300
3/3 [==============================] - ETA: 0s - loss: 0.4680 - accuracy: 0.8175
Epoch 109: val_loss did not improve from 0.70019
3/3 [==============================] - 1s 401ms/step - loss: 0.4680 - accuracy: 0.8175 - val_loss: 0.7240 - val_accuracy: 0.7101 - lr: 6.4000e-04
Epoch 110/300
3/3 [==============================] - ETA: 0s - loss: 0.4850 - accuracy: 0.8248
Epoch 110: val_loss did not improve from 0.70019
3/3 [==============================] - 1s 396ms/step - loss: 0.4850 - accuracy: 0.8248 - val_loss: 0.7356 - val_accuracy: 0.6957 - lr: 6.4000e-04
Epoch 111/300
3/3 [==============================] - ETA: 0s - loss: 0.5449 - accuracy: 0.7956
Epoch 111: val_loss did not improve

...

3/3 [==============================] - ETA: 0s - loss: 0.6047 - accuracy: 0.7591
Epoch 163: val_loss did not improve from 0.68938
3/3 [==============================] - 1s 401ms/step - loss: 0.6047 - accuracy: 0.7591 - val_loss: 0.7486 - val_accuracy: 0.7174 - lr: 2.6214e-04
Epoch 164/300
3/3 [==============================] - ETA: 0s - loss: 0.5847 - accuracy: 0.7664
Epoch 164: val_loss did not improve from 0.68938
3/3 [==============================] - 1s 393ms/step - loss: 0.5847 - accuracy: 0.7664 - val_loss: 0.7466 - val_accuracy: 0.7174 - lr: 2.6214e-04
Epoch 165/300
3/3 [==============================] - ETA: 0s - loss: 0.5795 - accuracy: 0.7883
Epoch 165: val_loss did not improve from 0.68938
3/3 [==============================] - 1s 408ms/step - loss: 0.5795 - accuracy: 0.7883 - val_loss: 0.7591 - val_accuracy: 0.7246 - lr: 2.6214e-04
Epoch 166/300
3/3 [==============================] - ETA: 0s - loss: 0.5752 - accuracy: 0.7883
Epoch 166: val_loss did not improve from 0.68938


3/3 [==============================] - 1s 405ms/step - loss: 0.5048 - accuracy: 0.8029 - val_loss: 0.8079 - val_accuracy: 0.6449 - lr: 1.3422e-04
Epoch 191/300
3/3 [==============================] - ETA: 0s - loss: 0.5137 - accuracy: 0.7883
Epoch 191: val_loss did not improve from 0.68938
3/3 [==============================] - 1s 401ms/step - loss: 0.5137 - accuracy: 0.7883 - val_loss: 0.8192 - val_accuracy: 0.6377 - lr: 1.3422e-04
Epoch 192/300
3/3 [==============================] - ETA: 0s - loss: 0.4882 - accuracy: 0.8029
Epoch 192: val_loss did not improve from 0.68938
3/3 [==============================] - 1s 391ms/step - loss: 0.4882 - accuracy: 0.8029 - val_loss: 0.8193 - val_accuracy: 0.6377 - lr: 1.3422e-04
Epoch 193/300
3/3 [==============================] - ETA: 0s - loss: 0.4886 - accuracy: 0.8029
Epoch 193: val_loss did not improve from 0.68938
3/3 [==============================] - 1s 396ms/step - loss: 0.4886 - accuracy: 0.8029 - val_loss: 0.8197 - val_accuracy: 0.6377 -

Epoch 218/300
3/3 [==============================] - ETA: 0s - loss: 0.4870 - accuracy: 0.8029
Epoch 218: val_loss did not improve from 0.68938
3/3 [==============================] - 1s 408ms/step - loss: 0.4870 - accuracy: 0.8029 - val_loss: 0.8132 - val_accuracy: 0.6522 - lr: 6.8719e-05
Epoch 219/300
3/3 [==============================] - ETA: 0s - loss: 0.4599 - accuracy: 0.8394
Epoch 219: val_loss did not improve from 0.68938
3/3 [==============================] - 1s 390ms/step - loss: 0.4599 - accuracy: 0.8394 - val_loss: 0.8127 - val_accuracy: 0.6522 - lr: 6.8719e-05
Epoch 220/300
3/3 [==============================] - ETA: 0s - loss: 0.4963 - accuracy: 0.8102
Epoch 220: val_loss did not improve from 0.68938
3/3 [==============================] - 1s 395ms/step - loss: 0.4963 - accuracy: 0.8102 - val_loss: 0.8120 - val_accuracy: 0.6522 - lr: 6.8719e-05
Epoch 221/300
3/3 [==============================] - ETA: 0s - loss: 0.4849 - accuracy: 0.8102
Epoch 221: val_loss did not improve

Epoch 246/300
3/3 [==============================] - ETA: 0s - loss: 0.4487 - accuracy: 0.8394
Epoch 246: val_loss did not improve from 0.68938

Epoch 246: ReduceLROnPlateau reducing learning rate to 3.518437442835421e-05.
3/3 [==============================] - 1s 438ms/step - loss: 0.4487 - accuracy: 0.8394 - val_loss: 0.8376 - val_accuracy: 0.6377 - lr: 4.3980e-05
Epoch 247/300
3/3 [==============================] - ETA: 0s - loss: 0.4411 - accuracy: 0.8321
Epoch 247: val_loss did not improve from 0.68938
3/3 [==============================] - 1s 439ms/step - loss: 0.4411 - accuracy: 0.8321 - val_loss: 0.8378 - val_accuracy: 0.6377 - lr: 3.5184e-05
Epoch 248/300
3/3 [==============================] - ETA: 0s - loss: 0.4733 - accuracy: 0.8248
Epoch 248: val_loss did not improve from 0.68938
3/3 [==============================] - 1s 433ms/step - loss: 0.4733 - accuracy: 0.8248 - val_loss: 0.8379 - val_accuracy: 0.6377 - lr: 3.5184e-05
Epoch 249/300
3/3 [==============================] 

3/3 [==============================] - 1s 457ms/step - loss: 0.4686 - accuracy: 0.8321 - val_loss: 0.8236 - val_accuracy: 0.6449 - lr: 2.2518e-05
Epoch 274/300
3/3 [==============================] - ETA: 0s - loss: 0.4837 - accuracy: 0.8102
Epoch 274: val_loss did not improve from 0.68938
3/3 [==============================] - 1s 542ms/step - loss: 0.4837 - accuracy: 0.8102 - val_loss: 0.8234 - val_accuracy: 0.6449 - lr: 2.2518e-05
Epoch 275/300
3/3 [==============================] - ETA: 0s - loss: 0.4421 - accuracy: 0.8540
Epoch 275: val_loss did not improve from 0.68938
3/3 [==============================] - 1s 514ms/step - loss: 0.4421 - accuracy: 0.8540 - val_loss: 0.8232 - val_accuracy: 0.6449 - lr: 2.2518e-05
Epoch 276/300
3/3 [==============================] - ETA: 0s - loss: 0.4634 - accuracy: 0.8175
Epoch 276: val_loss did not improve from 0.68938

Epoch 276: ReduceLROnPlateau reducing learning rate to 1.8014399392995985e-05.
3/3 [==============================] - 1s 455ms/st